# Trattare i Missing Values
## Introduzione

Il trattamento e l'imputazione dei valori mancanti (missing values) è uno step molto delicato per ogni progetto di data science.

Esistono diverse strategie per l'imputazione e tutte possono portare a errori perchè si sta introducendo un dato **"artificiale"**.
>Un consiglio che viene dato spesso è, in fase di imputazione di valori mancanti creare per ogni feature che si tratta una nuova variabile booleana "*nomeFeature_isMissing*" per tracciare quali valori sono reali e quali indotti durante il processo di cleaning.

Solitamente gli step che vengono seguiti in questa fase del preprocessing del dataset sono i seguenti:
* Se la percentuale di valori mancanti è alta (la soglia varia a seconda del contesto) non si può considerare la variabile, quindi viene eliminata uan feature
* Non eliminare mai l'intera osservazione a meno che non abbia valori mancanti su ogni feature
* Scegliete una tecnica di imputazione basandovi sul tipo di dato e fenomeno che si sta trattando

Lo strumento Python più utilizzato per questo compito è il [SimpleImputer di scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) che offre quattro strategie di sostituzione:
1. sostituire i missing values con la media
2. sostituire i missing values con la mediana
3. sostituire i missing values con la moda
4. sostituire i missing values con una costante

C'è una cosa da osservare, nessuna delle strategie descritte si adatta bene allo studio delle serie storiche. Per *dimostrare* questa affermazione faremo un test utilizzando la serie storica [**Airpassenger**](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/AirPassengers.html), una sequenza mensile del numero di passeggeri sui voli internazionali tra il 1949 e il 1960.

## Creazione di Missing Values

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Per prima cosa carichiamo la serie e vediamo:
* i primi 5 valori
* il plot dell'originale

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv", index_col = 0 )
airpassengers.columns = ["Passengers"]
airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:red')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

Successivamente generiamo una serie di 10 interi random. Sono le posizioni in cui andremo ad inserire i missing values. Infine vediamo il plot della nuova serie.

In [ ]:
airpassengers_MV = airpassengers.copy()

La funzione che genera gli indici per i missing value è casuale quindi ad ogni utilizzo del notebook la loro posizione cambierà.

In [ ]:
na_index = np.random.randint(0, airpassengers.shape[0]-1, 10)

In [ ]:
airpassengers_MV.iloc[na_index,0] = np.nan

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_MV["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing Values", xlabel="Time", ylabel="Passengers")
plt.show()

## Imputazione dei missing values

Di seguito verranno testate sulla serie tutte le strategie proposte da SimpleImputer di Scikit-learn.

### Sostituire i valori con la media

In [ ]:
airpassengers_mean = airpassengers_MV.copy()

In [ ]:
airpassengers_MV["Passengers"].mean()

In [ ]:
airpassengers_mean.iloc[na_index,0] = airpassengers_MV["Passengers"].mean()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_mean["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con la media", xlabel="Time", ylabel="Passengers")
plt.show()

Questo approccio risente molto del trend, Airpassenger è una serie con trend lineare crescente e un effetto stagionale moltiplicativo (le *onde* sono sempre più alte ogni anno), l'approccio dell'imputazione utilizzando la media crea picchi all'inizio della sequenza e valli sul finire, anche se queste ultime sono molto meno evidenti dei primi. Se la serie fosse ancora più lunga questo fenomeno sarebbe sempre più marcato.

### Sostituire i valori con la mediana

La mediana è una misura più robusta ai valori anomali rispetto alla media ma in questo caso produrrà lo stesso fenomeno. Questo è dovuto alla natura del dato, una serie con trend crescente e stagionalità.

In [ ]:
airpassengers_median = airpassengers_MV.copy()

In [ ]:
airpassengers_MV["Passengers"].median()

Essendo più bassa della media, la mediana produrrà picchi meno vistosi e valli più profonde.

In [ ]:
airpassengers_median.iloc[na_index,0] = airpassengers_MV["Passengers"].median()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_median["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con la mediana", xlabel="Time", ylabel="Passengers")
plt.show()

**Le strategie con media e mediana possono essere chiaramente scartate per l'imputazione di missing values nelle serie storiche**

### Sostituire i valori con il più frequente

In statistica il valore più frequente in una variabile si chiama MODA. Questa è una strategia ottima per variabili categoriali, sicuramente non per le serie storiche. Vediamo cosa succede.

In [ ]:
airpassengers_mode = airpassengers_MV.copy()

In [ ]:
airpassengers_MV["Passengers"].mode()[0]

Un valore a 229 potrebbe far passare inosservati i picchi. Il trend cresce linearmente e la stagionalità ha un effetto moltiplicativo, la moda in questo caso è un valore più basso della media, ma anche della mediana. Sembra dare un andamento quasi naturale alla serie nella parte centrale, ma quando il valore mancante si trova nei primi/ultimi periodi l'errore sistematico è evidente.

In [ ]:
airpassengers_mode.iloc[na_index,0] = airpassengers_MV["Passengers"].mode()[0]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_mode["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con la moda", xlabel="Time", ylabel="Passengers")
plt.show()

**N.B. Questa strategia, "Most Frequent", è molto utile se si hanno missing in variabili categoriali, un'altra strategia in questi casi può essere applicare un modello di Machine Learning (es. KNN) usando come target la variabile in cui dobbiamo sostituire i missing values e come features tutte le altre. 
<br>
Creare il training set per allenare il modello sui non missing e applicarlo sul test, ovvero le osservazioni che presentano i missing values**

### Sostituire i valori con una costante

In questo caso il problema che si pone immediatamente è *che costante utilizzare?*

#### Sostituire con zero

Lo zero solitamente è sconsigliatissimo perchè non si capisce se è assenza di valore o presenza pari a zero, crea molta abiguità.

>*Esempio: presenza ad eventi, se viene effettuata la sotituzione con zero non si riesce più a riconoscere se è un dato imputato o effettivamente c'è presenza zero (in aggiunta al restante errore indotto).*

In [ ]:
airpassengers_costant = airpassengers_MV.copy()

In [ ]:
airpassengers_costant.iloc[na_index,0] = 0

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_costant["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con zero", xlabel="Time", ylabel="Passengers")
plt.show()

#### Sostituire con il valore massimo nella serie

In [ ]:
airpassengers_costant.iloc[na_index,0] = airpassengers_MV.max()[0]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_costant["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con il massimo", xlabel="Time", ylabel="Passengers")
plt.show()

#### Sostituire con il valore minimo nella serie

In [ ]:
airpassengers_costant.iloc[na_index,0] = airpassengers_MV.min()[0]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_costant["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti con il minimo", xlabel="Time", ylabel="Passengers")
plt.show()

Come si intuisce immediatamente tutte e tre le strategie sono fallimentari.

### Sostituire i valori mediante Media Mobile

La sostituzione dei valori utilizzando la [Media Mobile](https://it.wikipedia.org/wiki/Media_mobile) permette di imputare al valore mancante la media locale in un range deciso dall'analista.

Data un serie storica ${y_t}$ con $t=(1, 2, ..., T)$, sia un generico elemento della serie $t$ un valore mancante e data una finestra temporale di dimensione $N$, siano:

* $m_1$ gli $N$ periodi antecedenti il valore mancante
* $m_2$ gli $N$ periodi successivi al valore mancante
* $\theta_i$ il peso da attribuire all'i-esimo valore osservato. (*Per noi sarà pari ad 1 visto che vogliamo una media aritmetica semplice*).

Si definisce media mobile al tempo $t$:

$mm_i$ = $\frac{1}{k}$ $\sum_{i=-m_1}^{m_2}$ $\theta_i$ $y_{t+1}$

Dove $k = {m_1} + {m_2} + 1$

<br>

Il metodo è un pò più esoso in termini di calcolo e presenta alcuni problemi che vanno risolti, ad esempio:
* se manca il valore all'inizio o alla fine della serie?
* se nel range indicato ci sono più valori mancanti?
* se abbiamo missing values contigui?

Fortunatamente in questo esempio non si presentano molti di questi, ma sta al data scientist scegliere quale soluzione applicare ad ogni domanda a seconda del contesto.
<br>
Nel nostro caso applichiamo un'interpolazione tra i tre valori prima e i tre valori dopo il missing value, gestendo i due casi:
1. Il lower bound è negativo
2. L'upper bound supera la lunghezza della serie.

In [ ]:
airpassengers_MA = airpassengers_MV.copy()
steps = 3

In [ ]:
for idx in na_index:
    lower = idx - steps
    upper = idx + steps + 1
    if lower<0:
        lower=0
    if upper>airpassengers_MA.shape[0]:
        upper=airpassengers_MA.shape[0]
    
    airpassengers_MA.iloc[idx,0] = airpassengers_MA.iloc[lower:upper,0].mean()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers_MA["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers - Missing values sostituiti per Media Mobile", xlabel="Time", ylabel="Passengers")
plt.show()

Confrontando i risultati con la serie originale si può notare che questa soluzione è molto più performante delle precedenti.
* Rosso: serie Originale
* Blu: serie con trattamento dei missing values

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:red')
plt.plot(range(airpassengers.shape[0]), airpassengers_MA["Passengers"].values, color='tab:blue')
plt.gca().set(title="CONFRONTO: Airpassengers - Missing values sostituiti per Media Mobile", xlabel="Time", ylabel="Passengers")
plt.show()

**Imputare un valore mancante introduce sempre errore sistematico, si può solo scegliere la strategia più adatta per minimizzarlo.**